### 네이버 쇼핑 키워드 인사이트

#### 개인과제
- 키워드에 대한 검색량 데이터 수집
- 검색어에 대한 상품등록수 수집
- 수집한 데이터에 대한 인사이트 도출 : 예시) 경쟁률

#### 조별과제
- 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드와 상품 찾기
    - 예시외의 추가적인 페이지 데이터 수집하여 프로젝트 수행가능

#### 참고 서비스
- 신사임당 유투브 : https://youtu.be/Z0hL9khbpIw
- 응용 서비스 : https://whereispost.com/keyword, https://itemscout.io

#### 데이터 수집 프로세스
1. 네이버 검색광고 회원가입 및 로그인 : https://searchad.naver.com
2. 광고 시스템 > 도구 > API 사용관리
3. CUSTOMER_ID, API_KEY, SECRET_KEY 획득
4. 네이버 검색 광고 API 매뉴얼 사이트 참고 : http://naver.github.io/searchad-apidoc/#/guides
5. github 페이지 참고
    - https://github.com/naver/searchad-apidoc
    - https://github.com/naver/searchad-apidoc/blob/master/python-sample/examples/ad_management_sample.py
6. 키워드 입력하면 해당 키워드에 대한 데이터 수집

- hint
    - github 페이지의 코드에서 signaturehelper.py 파일을 이용하여 header를 만들어야 합니다.

In [1]:
import requests, json, time
import pandas as pd
import signaturehelper
import warnings
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

#### 1. 생성된 key를 변수에 저장

In [2]:
BASE_URL = 'https://api.naver.com'
API_KEY = '01000000002e4be86c2682ead4b17add22f9609aa03730d356e9f0f4a3cde779abdceb3e2a'
SECRET_KEY = 'AQAAAAAuS+hsJoLq1LF63SL5YJqgKZ95ZgSX98xuJvZibX1CxA=='
CUSTOMER_ID = '2488401'

#### 2. header 함수 생성

In [3]:
import hashlib, hmac, base64, requests, time

def generate(timestamp, method, uri, secret_key):
    message = "{}.{}.{}".format(timestamp, method, uri)
#     hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)
    hash = hmac.new(secret_key.encode("utf-8"), message.encode("utf-8"), hashlib.sha256)

    hash.hexdigest()
    return base64.b64encode(hash.digest())

def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(int(time.time() * 1000))
    signature = generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

#### 1. 개별과제 : 키워드에 대한 검색량 데이터 수집

In [4]:
def keyword_master(keyword):
    # 1. 웹서비스 분석: URL
    uri = '/keywordstool'
    method = 'GET'
    prm = {'hintKeywords' : keyword, 'showDetail':1}
    
    # 2. request(url) : url, params, header
    r = requests.get(BASE_URL + uri, params=prm, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
    r_data = r.json()['keywordList'][0]
    df=pd.DataFrame(r.json()['keywordList'])
    
    return df

In [5]:
keyword = "실내자전거"
df = keyword_master(keyword)

In [6]:
df.head()

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,실내자전거,19400,163400,112.1,2643.0,0.66,1.76,15,높음
1,하체운동기구,1070,7790,8.0,120.8,0.81,1.66,15,높음
2,헬스자전거,1240,11900,17.4,318.0,1.56,2.86,15,높음
3,스피닝자전거,2330,24200,12.5,360.5,0.58,1.56,15,높음
4,스쿼트머신,2260,16800,5.0,195.0,0.24,1.23,15,높음


#### 2. 개별과제 : 검색어에 대한 상품등록수 수집
- item_count : 등록된 상품수
- total_search_count : PC, Mobile 을 합친 검색량

In [7]:
df2 = df.iloc[:,[0,1,2]]
df2

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt
0,실내자전거,19400,163400
1,하체운동기구,1070,7790
2,헬스자전거,1240,11900
3,스피닝자전거,2330,24200
4,스쿼트머신,2260,16800
...,...,...,...
1005,종아리스트레칭기구,140,1360
1006,발목경사대,110,920
1007,종아리운동기구,90,1420
1008,종아리운동,450,3230


#### 클라이언트 아이디, 클라이언트 시크릿

In [10]:
client_id = "aQvBJ43Ls53JgtglTDAC"# Your client_id
client_secret = "eN7FPg_oaj" # Your client_secret

In [38]:
import requests
import urllib

def item_count(keyword):

    keyword = urllib.parse.quote(keyword)

    url = "https://openapi.naver.com/v1/search/shop?query=" + keyword

    request = urllib.request.Request(url)
    request.add_header('X-Naver-Client-Id', client_id)
    request.add_header('X-Naver-Client-Secret', client_secret)

    response = urllib.request.urlopen(request)

    data = response.read().decode('utf-8')
    jdata = json.loads(data)
    
    return jdata["total"]

In [39]:
keyword = "실내자전거"
print(item_count(keyword))

255088


In [ ]:
for idx in df.index:

#### 3. 개별과제 : 경쟁률
- 아이템수/검색수 : 높을수록 경쟁이 심한 상품 키워드
- 수집된 데이터에서 추가로 얻을수 있는 인사이트 추가
    - 논리적인 인사이트이면 인사이트 수준에 대한 평가는 하지 않음

#### 4. 조별과제 : 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드 찾기
- 정답이 있는것은 아닙니다
- 여러가지 가정과 데이터를 수집, 분석하여 판매할 상품과 키워드를 선정해 보시면 됩니다.
- 앞서 2일간 했던 프로젝트와 같은 방법으로 수행하시면 됩니다.

### 평가기준

#### 개인평가(70)

    - 기본 문제 해결(30)
        - 데이터 수집 방법 : 10점
            - 최적의 수집 방법을 사용했는지 확인
        - 수집된 데이터의 결과 : 20점
            - 1번: 키워드에 대한 검색량 데이터 수집 : 15점
            - 2번: 검색어에 대한 상품등록수 수집 : 5점
            
    - 적절한 주석 및 설명(10)
        - 코드의 가독성 및 코드의 설명 : 10점
        
    - 코드 오류(10)
        - 발생 오류마다 : -2점
        
    - 추가 분석(20)
        - 인사이트 도출
            - 경쟁률 데이터 출력 : 10점
            - 추가 인사이트 : 10점
            
#### 조별평가(30)
    
    - 비지니스 인사이트 도출(5)
        - 스마트 스토어 운영자 입장에서의 추가 인사이트 도출
        - 예시) 선정한 상품으로 소호몰에서 1위 하였을경우 예상되는 매출
        - 인사이트 도출을 위한 프로세스
        
    - 실습문제의 분석(5)
        - 제출 자료가 잘 정리되고 판매 상품과 키워드를 논리적으로 잘 선정
        - 상품과 키워드 선정 자동화 > 가독성 좋은 키워드와 상품 선정 프로세스
        
    - 실습문제의 적절한 정리(20)
        - 프로젝트 주제(상품과 키워드 선정)에 맞는 논리적 인사이트가 있는지 확인
        - 프로젝트 수행 후 추후 과제 및 느낀점